In [1]:
import transx2gtfs

frompath = "C:/Users/Lukas/OneDrive/Documents/Philita/London TransXChange tube only/LULDLRTRAMRIVERCABLE 10062020 EDIT.zip"
topath = "C:/Users/Lukas/OneDrive/Documents/Philita/London GTFS/my_converted_gtfs.zip"

transx2gtfs.convert(frompath, topath)


Populating database ..


BadZipFile: File is not a zip file

In [ ]:
# SAVING SHIT
'''
Created on 5 Jun 2020

@author: lukas

HTTP requests and dictionary returns from OTP2
'''

# Imports 
import sys
import pandas as pd
import requests
import time
import psutil
import subprocess
import numpy as np

# Increase the size of the display. Just for me :)
# from IPython.display import display, HTML

# display(HTML(data="""
# <style>
#     div#notebook-container    { width: 90%; }
#     div#menubar-container     { width: 65%; }
#     div#maintoolbar-container { width: 99%; }
# </style>
# """))

""" Meat and Podaydoes """

# Print system version
print(sys.version)

# Define the names we are searching for. Can be taken from the Popen commands
collection = ['otp-2.0.0-SNAPSHOT-shaded.jar']

# Reset the PID
PID = None

# Check if OTP is running. If it is store the PID. Is this MS Windows specific? 
for process in psutil.process_iter():
    try:
        for x in process.cmdline():
                if x in collection:
                    print("The OTP has been already been started")
                    PID = process.pid
    except:
        continue 
        
# Instantiate the OTP if it has not been started already.
if PID == None:
    subprocess.Popen('cd "C:/Users/Lukas/OneDrive/Documents/Philita/OpenTripPlanner2/target" & java -Xmx8G -jar otp-2.0.0-SNAPSHOT-shaded.jar --build --serve "C:/Users/Lukas/OneDrive/Documents/Philita/Source for OTP" ', shell=True)
    print("Starting the OTP")

# Required
i = 0    

# Wait until the OTP graph is built and the server is launched
while i < 50:
    try:
        r = requests.get('http://localhost:8080/')
        if r:
            print("The server has launched correctly")
            i = 9999
        else: 
            print("The server has not started correctly")
            time.sleep(10)
            i += 1            
    except:
        print("The server is not reachable")
        time.sleep(10)
        i += 1  

if i == 18:
    print("The server has not started in 500 seconds. ERROR 001")
    sys.exit()
    
        
# Read variables as a Pandas dataframe
PopInput = pd.read_csv("C:/Users/Lukas/OneDrive/Documents/Philita/Source for OTP/PopulationInput.csv", index_col=0, dtype={"Time":str});       
 
# # Display the dataframe
# print("|||||||||||||||||||")
# print(PopInput)

# Create holding variables
i = 0
j = 0
StartLat = None 
StartLon = None
EndLat = None
EndLon = None
time = None
PopAmount = None

# Import the stop capacity csv
StopCapacity = pd.read_csv("C:/Users/Lukas/OneDrive/Documents/Philita/Source for OTP/stop_capacity_mod.csv",
                           dtype={"business":np.int32},
                           na_filter=False)


# Shared request variables
mode = "TRANSIT,WALK" # Transport modes to consider, in this case a combination of walking and transit
maxWalkDistance = 1000 # The maximum distance in meters that you are willing to walk
date = "06-15-2020" # The desired departure date. M-D-Y format
arriveBy = "false"  # Specifies that the given time is when we plan to depart rather than when we want to arrive
row_num = 1

# # The big loop. 
# First read the required info from PopInput
while row_num <= len(PopInput)-1: 
    StartLat = PopInput.loc["{}".format(row_num),"StartLat"]
    StartLon = PopInput.loc["{}".format(row_num),"StartLon"]
    EndLat = PopInput.loc["{}".format(row_num),"EndLat"]
    EndLon = PopInput.loc["{}".format(row_num),"EndLon"]
    time = PopInput.loc["{}".format(row_num),"Time"]
    PopAmount = PopInput.loc["{}".format(row_num),"PopAmount"]
    row_num += 1
    
    print("|||||||||||||||||||")
    print(PopAmount)    


#     Then pass the request to the server.  
    TripDict = requests.get("http://localhost:8080/otp/routers/default/plan?fromPlace={},{}\
                            &toPlace={},{}\
                            &time={}\
                            &date={}\
                            &mode=TRANSIT,WALK\
                            &maxWalkDistance=500\
                            &arriveBy=false"\
                            .format(StartLat,StartLon,EndLat,EndLon,time,date,arriveBy,mode,maxWalkDistance)
                            )        

    # Then recieve the output trip dictionary and parse as JSON
    route = TripDict.json()

    # Create some required variables
    holding = ["to", "from"]
    itinerary_no = len(route["plan"]["itineraries"])
    i = 0
    loading = []

    # Get the required info from the trip dictionary and add to a collector. Cleans the strings by removing an internal OTP identifier.
    while i < itinerary_no: 
        for step in holding:
            clean = route["plan"]["itineraries"][i]["legs"][1][step]["stopId"]
            loading.append(clean[2:])
        i += 1
    
#     print("|||||||||||||||||||")
#     print(loading)    

    # Returns the row number of the common values in a list 
    contained = [StopCapacity[StopCapacity["stop_id"] == x].index[0] for x in StopCapacity["stop_id"] if x in loading]  

#     print("|||||||||||||||||||")
#     print(contained)

    print("|||||||||||||||||||")
    # Add 10 to the business column if the stop_id is matched.     
    for row in contained:
        StopCapacity.iloc[row,5] = StopCapacity.iloc[row,5] + 10

    
    
    
    
# Save the pd.DataFrame into the CSV. Overwrite the old one. 
StopCapacity.to_csv("C:/Users/Lukas/OneDrive/Documents/Philita/Source for OTP/stop_capacity_mod.csv")
    
    
    
    
# Whether we want to kill the OTP instance upon finishing the script
Kill = True     
    
# If the kill check is true kill the process
if Kill:
    if PID is not None:
        subprocess.run("taskkill  /F  /PID  {}".format(PID))
        print("The OTP has been shut down")
